In [1]:
import pandas as pd
import shap
data = pd.read_csv("../../samples/lidl_train.csv")
data.dropna(inplace=True)

/home/andre/arbeit/sentiment-in-marketing/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
low_rated = """Quit it with the forced updates . Yet again today , in the queue turned it on , update.. all i wanted was to get a digital reciept for electrical purchases, but no wouldnt let me scan. Judt update. At the checkout with lousy mobile coverage is not the place to find out there is an update out..thanks for your answer i understand apps need updating . But its the forced nature of your updates.. ""do nothing until its updated"" at least let us scan the code as a minimum.before updating."""
low_score = 0

high_rated = """It's a good app and the savings are better than other loyalty schemes. The scratch card seems a bit of a gimmick but otherwise it's a good app and recommended. Also shows how busy the shop is and opening hours 👍"""
high_score = 4

# data = pd.DataFrame({"review": [low_rated, high_rated], "rating": [low_score, high_score]})
data

,rating,review,date
0,5,"Always have what I need, fresh good quality fo...",2022-05-30
1,5,Excellent,2023-10-27
2,5,Great,2021-12-22
3,1,Very deceptive sales. Make sure to read all of...,2023-12-22
4,4,Easy to use,2022-09-30
...,...,...,...
27235,5,Extremely good value for money with a wonderfu...,2022-06-07
27236,4,Always look forward to money off coupons each ...,2023-01-12
27237,5,Good,2022-09-16
27238,5,excellent,2024-02-09


In [3]:
from joblib import load
from sklearn.pipeline import Pipeline
# model = load("./results/train/MultinomialNB/MultinomialNB_lidl_2024-04-03_11-55-30.pkl")
model = load("../../results/train/MultinomialNB/MultinomialNB_lidl_2024-04-03_11-55-30.pkl")
# shap does not work with the grid search model
pipeline = model.best_estimator_
clf = pipeline.named_steps["clf"]
vect = pipeline.named_steps["vect"]
tfidf = pipeline.named_steps["tfidf"]
pipeline = Pipeline([("vect", vect), ("tfidf", tfidf)])

In [4]:
X_train = pipeline.transform(data["review"]).toarray()
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [5]:
prediction = clf.predict(X_train)[0]
predicted_class_idx = list(clf.classes_).index(prediction)

In [6]:
pipeline.transform([high_rated]).toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

In [9]:
#convert your training and testing data using the TF-IDF vectorizer
import matplotlib


tfidf_vectorizer = pipeline
tfidf_train = tfidf_vectorizer.transform(data.review) 
tfidf_test = tfidf_vectorizer.transform([high_rated])


model=clf

#shap apply
#first shorten the data & convert to data frame
X_test_sample = tfidf_test

X_train_sample = tfidf_train
X_train_sample = shap.sample(X_train_sample, 50)
sample_text = shap.sample(tfidf_test, 1)
 
SHAP_explainer = shap.KernelExplainer(model.predict, X_train_sample)
shap_vals = SHAP_explainer.shap_values(sample_text)
print(SHAP_explainer.expected_value)
shap.initjs() # required for visualizations
# shap_plot = shap.force_plot(SHAP_explainer.expected_value, shap_vals,sample_text.toarray(), feature_names=tfidf_vectorizer.get_feature_names_out(), matplotlib=True) # works but not nicely
# shap_plot = shap.force_plot(SHAP_explainer.expected_value, shap_vals,sample_text.toarray(), feature_names=tfidf_vectorizer.get_feature_names_out()) # works but not ideal (too many features)
shap.plots.force(SHAP_explainer.expected_value, shap_vals[0], sample_text.toarray(), contribution_threshold=0)
# shap.plots.force(SHAP_explainer.expected_value, shap_vals, sample_text.toarray(), contribution_threshold=0.4, feature_names=tfidf_vectorizer.get_feature_names_out()) # works but visually cluttered
# shap_text = shap.text_plot(shap_vals[0])
# shap.plots.text(shap_vals)
# shap.force_plot(SHAP_explainer.expected_value, shap_vals,sample_text.toarray(), feature_names=tfidf_vectorizer.get_feature_names_out())
# shap.force_plot(SHAP_explainer.expected_value, shap_vals,sample_text.toarray())

100%|██████████| 1/1 [00:02<00:00,  2.94s/it]

2.7800000000000002


In [8]:
shap_plot

NameError: name 'shap_plot' is not defined

In [ ]:
shap_plot

In [ ]:
tfidf_vectorizer.get_feature_names_out()

In [ ]:
shap_plot

In [ ]:
#convert your training and testing data using the TF-IDF vectorizer
tfidf_vectorizer = pipeline
tfidf_train = tfidf_vectorizer.transform(data.review)
tfidf_test = tfidf_vectorizer.transform([high_rated])
import numpy as np
from scipy import sparse

model=clf

#shap apply
#first shorten the data & convert to data frame
X_test_sample = tfidf_test

X_train_sample = tfidf_train
X_train_sample = shap.sample(X_train_sample, 50)
sample_text = shap.sample(tfidf_test, 1)

Reference_value=np.zeros(np.shape(tfidf_test)[1])
RV=np.reshape(Reference_value,(1,len(Reference_value)))
RV=sparse.csr_matrix(RV)
 
# SHAP_explainer = shap.KernelExplainer(model.predict_proba, X_train_sample, link="identity")
SHAP_explainer = shap.KernelExplainer(model.predict_proba, RV, link="identity")
shap_vals = SHAP_explainer(sample_text)
print(SHAP_explainer.expected_value)
# shap_text = shap.text_plot(shap_vals[0,:,1])
shap_text = shap.text_plot(shap_vals)

In [ ]:
sample_text

In [ ]:
X_train_sample

In [ ]:
#convert your training and testing data using the TF-IDF vectorizer
tfidf_vectorizer = pipeline
tfidf_train = tfidf_vectorizer.transform(data.review) 
tfidf_test = tfidf_vectorizer.transform([high_rated])


model=clf

#shap apply
#first shorten the data & convert to data frame
X_test_sample = tfidf_test

X_train_sample = tfidf_train
X_train_sample = shap.sample(X_train_sample, 50)
sample_text = shap.sample(tfidf_test, 1)
 
SHAP_explainer = shap.KernelExplainer(model.predict, X_train_sample, link="identity")
shap_vals = SHAP_explainer(sample_text)
print(SHAP_explainer.expected_value)
# shap_plot = shap.force_plot(SHAP_explainer.expected_value, shap_vals,sample_text.toarray(), feature_names=tfidf_vectorizer.get_feature_names_out())
shap_text = shap.text_plot(shap_vals, sample_text.toarray(), tfidf_vectorizer.get_feature_names_out())
# print(shap_vals)
# shap.plots.text(shap_vals)

In [ ]:
sample_text.toarray().dtype